# Ζητούμενο 2

### α) Να υλοποιηθεί το Query 2 χρησιμοποιώντας τα DataFrame και SQL APIs. Να αναφέρετε και να συγκρίνετε τους χρόνους εκτέλεσης μεταξύ των δύο υλοποιήσεων. 

#### Προεπεξεργασία για το query 2: Ενώνουμε τα 2 μέρη του κυρίως data set (Los Angeles Crime Data) σε ένα ενιαίο dataframe και στην συνέχεια το αποθηκεύουμε σε ένα μοναδικό .csv αρχείο στο S3 bucket του group 5, έτσι ώστε η μετέπειτα σύγκριση να γίνει υπό δίκαιες συνθήκες!

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, DoubleType, StringType
from pyspark.sql.functions import col


spark = SparkSession \
    .builder \
    .appName("DF query 1 execution") \
    .getOrCreate()

crime_data_schema = StructType([StructField("dr_no", IntegerType(), False),
    StructField("date_rptd", StringType(), True),
    StructField("date_occ", StringType(), True),
    StructField("time_occ", IntegerType(), True),
    StructField("area", IntegerType(), True),
    StructField("area_name", StringType(), True),
    StructField("rpt_dist_no", IntegerType(), True),
    StructField("part_1_2", IntegerType(), True),
    StructField("crm_cd", IntegerType(), True),
    StructField("crm_cd_desc", StringType(), True),
    StructField("mocodes", StringType(), True),
    StructField("vict_age", IntegerType(), True),
    StructField("vict_sex", StringType(), True),
    StructField("vict_descent", StringType(), True),
    StructField("premis_cd", IntegerType(), True),
    StructField("premis_desc", StringType(), True),
    StructField("weapon_used_cd", IntegerType(), True),
    StructField("weapon_desc", StringType(), True),
    StructField("status", StringType(), True),
    StructField("status_desc", StringType(), True),
    StructField("crm_cd_1", IntegerType(), True),
    StructField("crm_cd_2", IntegerType(), True),
    StructField("crm_cd_3", IntegerType(), True),
    StructField("crm_cd_4", IntegerType(), True),
    StructField("location", StringType(), True),
    StructField("cross_street", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True)])


crime_data_10_to_19_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(crime_data_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv")

crime_data_20_to_present_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(crime_data_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv")

unified_crime_data_df = crime_data_10_to_19_df.union(crime_data_20_to_present_df)

group_number = "5"
base_s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/CrimeData/"
csv_s3_path = base_s3_path + "csv/"
unified_crime_data_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(csv_s3_path)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
253,application_1738075734771_0254,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Εκτέλεση του query 2, χρησιμοποιώντας DataFrame.

In [2]:
# DF query 2 execution
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, DoubleType, StringType
from pyspark.sql.functions import col, substring, count, round, dense_rank
from pyspark.sql.window import Window
import time


spark = SparkSession \
    .builder \
    .appName("DF query 2 execution") \
    .getOrCreate()

start_time = time.time()

unified_crime_data_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(crime_data_schema) \
    .load(csv_s3_path)

cases_with_year_df = unified_crime_data_df.withColumn("year", substring(col("date_occ"), 7, 4))

total_cases_by_station_year_df = cases_with_year_df.groupBy("year", "area_name") \
    .agg(count("*").alias("total_cases"))

#total_cases_by_station_year_df.show(22)

cases_closed_df = cases_with_year_df.filter(
    (~col("status_desc").contains("UNK")) & 
    (~col("status_desc").contains("Invest Cont")))

#cases_closed_df.show(5)

cases_closed_by_station_year_df = cases_closed_df.groupBy("year", "area_name") \
    .agg(count("*") \
    .alias("closed_cases"))

#cases_closed_by_station_year_df.show(20)

cases_with_percentages_df = cases_closed_by_station_year_df.join(total_cases_by_station_year_df, ["year", "area_name"]) \
    .withColumn("closed_case_rate",(col("closed_cases") / col("total_cases") * 100))

#cases_with_percentages.show(5)

window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())

query2_results_df = cases_with_percentages_df.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3) \
    .select("year", col("area_name").alias("precinct"), "closed_case_rate", "rank")

query2_results_df = query2_results_df.orderBy("year", "rank")

query2_results_df.show(truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|precinct   |closed_case_rate  |rank|
+----+-----------+------------------+----+
|2010|Rampart    |32.84713448949121 |1   |
|2010|Olympic    |31.515289821999087|2   |
|2010|Harbor     |29.36028339237341 |3   |
|2011|Olympic    |35.040060090135206|1   |
|2011|Rampart    |32.4964471814306  |2   |
|2011|Harbor     |28.51336246316431 |3   |
|2012|Olympic    |34.29708533302119 |1   |
|2012|Rampart    |32.46000463714352 |2   |
|2012|Harbor     |29.509585848956675|3   |
|2013|Olympic    |33.58217940999398 |1   |
|2013|Rampart    |32.1060382916053  |2   |
|2013|Harbor     |29.723638951488557|3   |
|2014|Van Nuys   |32.0215235281705  |1   |
|2014|West Valley|31.49754809505847 |2   |
|2014|Mission    |31.224939855653567|3   |
|2015|Van Nuys   |32.265140677157845|1   |
|2015|Mission    |30.463762673676303|2   |
|2015|Foothill   |30.353001803658852|3   |
|2016|Van Nuys   |32.194518462124094|1   |
|2016|West Valley|31.40146437042384 |2   |
+----+-----

#### Εκτέλεση του query 2, χρησιμοποιώντας SQL APIs.

In [3]:
# SQL API query 2 execution
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, DoubleType, StringType
from pyspark.sql.functions import col, substring, count, round, dense_rank
from pyspark.sql.window import Window
import time


spark = SparkSession \
    .builder \
    .appName("SQL API query 2 execution") \
    .getOrCreate()

start_time = time.time()

unified_crime_data_df = spark.read.format('csv') \
    .options(header='true') \
    .schema(crime_data_schema) \
    .load(csv_s3_path)

unified_crime_data_df.createOrReplaceTempView("unified_crime_data")

# Create a new table with the 'year' column extracted
spark.sql("""
    CREATE OR REPLACE TEMP VIEW cases_with_year AS
    SELECT *, SUBSTRING(date_occ, 7, 4) AS year FROM unified_crime_data
""")

# Combined query to compute closed case rates and rank top 3 areas per year
query2_results_sql = spark.sql("""
    WITH 
    total_cases_by_station_year AS (
        SELECT
            year,
            area_name,
            COUNT(*) AS total_cases
        FROM cases_with_year
        GROUP BY year, area_name
    ),
    cases_closed_by_station_year AS (
        SELECT
            year,
            area_name,
            COUNT(*) AS closed_cases
        FROM cases_with_year
        WHERE status_desc NOT LIKE '%UNK%'
          AND status_desc NOT LIKE '%Invest Cont%'
        GROUP BY year, area_name
    ),
    cases_with_percentages AS (
        SELECT
            c.year,
            c.area_name,
            c.closed_cases,
            t.total_cases,
            (c.closed_cases / t.total_cases * 100) AS closed_case_rate
        FROM cases_closed_by_station_year c
        JOIN total_cases_by_station_year t
        ON c.year = t.year AND c.area_name = t.area_name
    )
    SELECT 
        year,
        area_name AS precinct,
        closed_case_rate,
        rank
    FROM (
        SELECT 
            year,
            area_name,
            closed_case_rate,
            DENSE_RANK() OVER (PARTITION BY year ORDER BY closed_case_rate DESC) AS rank
        FROM cases_with_percentages
    ) ranked
    WHERE rank <= 3
    ORDER BY year, rank
""")

query2_results_sql.show(truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|precinct   |closed_case_rate  |rank|
+----+-----------+------------------+----+
|2010|Rampart    |32.84713448949121 |1   |
|2010|Olympic    |31.515289821999087|2   |
|2010|Harbor     |29.36028339237341 |3   |
|2011|Olympic    |35.040060090135206|1   |
|2011|Rampart    |32.4964471814306  |2   |
|2011|Harbor     |28.51336246316431 |3   |
|2012|Olympic    |34.29708533302119 |1   |
|2012|Rampart    |32.46000463714352 |2   |
|2012|Harbor     |29.509585848956675|3   |
|2013|Olympic    |33.58217940999398 |1   |
|2013|Rampart    |32.1060382916053  |2   |
|2013|Harbor     |29.723638951488557|3   |
|2014|Van Nuys   |32.0215235281705  |1   |
|2014|West Valley|31.49754809505847 |2   |
|2014|Mission    |31.224939855653567|3   |
|2015|Van Nuys   |32.265140677157845|1   |
|2015|Mission    |30.463762673676303|2   |
|2015|Foothill   |30.353001803658852|3   |
|2016|Van Nuys   |32.194518462124094|1   |
|2016|West Valley|31.40146437042384 |2   |
+----+-----

### β) Να γράψετε κώδικα Spark που μετατρέπει το κυρίως data set σε parquet file format και αποθηκεύει ένα μοναδικό .parquet αρχείο στο S3 bucket της ομάδας σας. Επιλέξτε μία από τις δύο υλοποιήσεις του υποερωτήματος α) (DataFrame ή SQL) και συγκρίνετε τους χρόνους εκτέλεσης της εφαρμογής σας όταν τα δεδομένα εισάγονται σαν .csv και σαν .parquet. 

#### Μετατροπή του κυρίως data set σε parquet file format και αποθήκευσή του σε ένα μοναδικό .parquet αρχείο στο S3 bucket του group 5.

In [4]:
group_number = "5"
base_s3_path = "s3://groups-bucket-dblab-905418150721/group"+group_number+"/CrimeData/"

parquet_s3_path = base_s3_path + "parquet/"
unified_crime_data_df.coalesce(1).write.mode("overwrite").parquet(parquet_s3_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Επιλέγουμε την υλοποίηση με DataFrame και εκτελούμε την εφαρμογή εισάγοντας τα δεδομένα σαν .parquet .

In [5]:
# DF query 2 execution and .parquet
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, DoubleType, StringType
from pyspark.sql.functions import col, substring, count, round, dense_rank
from pyspark.sql.window import Window
import time


spark = SparkSession \
    .builder \
    .appName("DF query 2 execution and .parquet") \
    .getOrCreate()

start_time = time.time()

unified_crime_data_df = spark.read.parquet(parquet_s3_path)

cases_with_year_df = unified_crime_data_df.withColumn("year", substring(col("date_occ"), 7, 4))

total_cases_by_station_year_df = cases_with_year_df.groupBy("year", "area_name") \
    .agg(count("*").alias("total_cases"))

cases_closed_df = cases_with_year_df.filter(
    (~col("status_desc").contains("UNK")) & 
    (~col("status_desc").contains("Invest Cont")))

cases_closed_by_station_year_df = cases_closed_df.groupBy("year", "area_name") \
    .agg(count("*") \
    .alias("closed_cases"))

cases_with_percentages_df = cases_closed_by_station_year_df.join(total_cases_by_station_year_df, ["year", "area_name"]) \
    .withColumn("closed_case_rate",(col("closed_cases") / col("total_cases") * 100))

window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())

query2_results_df = cases_with_percentages_df.withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3) \
    .select("year", col("area_name").alias("precinct"), "closed_case_rate", "rank")

query2_results_df = query2_results_df.orderBy("year", "rank")

query2_results_df.show(truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|precinct   |closed_case_rate  |rank|
+----+-----------+------------------+----+
|2010|Rampart    |32.84713448949121 |1   |
|2010|Olympic    |31.515289821999087|2   |
|2010|Harbor     |29.36028339237341 |3   |
|2011|Olympic    |35.040060090135206|1   |
|2011|Rampart    |32.4964471814306  |2   |
|2011|Harbor     |28.51336246316431 |3   |
|2012|Olympic    |34.29708533302119 |1   |
|2012|Rampart    |32.46000463714352 |2   |
|2012|Harbor     |29.509585848956675|3   |
|2013|Olympic    |33.58217940999398 |1   |
|2013|Rampart    |32.1060382916053  |2   |
|2013|Harbor     |29.723638951488557|3   |
|2014|Van Nuys   |32.0215235281705  |1   |
|2014|West Valley|31.49754809505847 |2   |
|2014|Mission    |31.224939855653567|3   |
|2015|Van Nuys   |32.265140677157845|1   |
|2015|Mission    |30.463762673676303|2   |
|2015|Foothill   |30.353001803658852|3   |
|2016|Van Nuys   |32.194518462124094|1   |
|2016|West Valley|31.40146437042384 |2   |
+----+-----